In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
wine_quality = fetch_ucirepo(id=186)

# data (as pandas dataframes)
X= wine_quality.data.features
Y = wine_quality.data.targets
X

In [ ]:
# x.iloc[1598]
# y.iloc[1598]
x = X.iloc[:1599]
y = Y.iloc[:1599]


In [ ]:
len(x)

In [ ]:
x.iloc[1598],y.iloc[1598]

In [ ]:
feature=[]
for i in x:
  print(i)
  feature.append(i)

In [ ]:
y

In [ ]:
feature

In [ ]:
y['quality'].value_counts()

**Convert all the values in the quality attribute to 0 (bad) if the value is less than or equal to 6,
and to 1 (good) otherwise. Normalise all the other attributes between 0 and 1 by min-max scaling.
Use this dataset (dataset A) for Logistic Regression**

In [ ]:
#  a=y['quality'].values
#  a
#  y=a

In [ ]:
a=y['quality'].values
a
y=a
for i in range(len(y)):
  if y[i]<6:
    y[i]=0
  else:
    y[i]=1

In [ ]:
one =0
zero=0
for i in y:
  if i==1:
    one+=1
  else:
    zero+=1
print(" 0 :",zero)
print(" 1 :",one)

In [ ]:
data=[]
for i in range(len(feature)):
  xi=x[feature[i]].values
  data.append(xi)

In [ ]:
len(data),len(feature),len(data[0])

In [ ]:
data[0][3],data[4][0]

In [ ]:
for i in range(len(data)):
  z=data[i]
  mn=min(z)
  mx=max(z)
  diff=mx-mn
  for j in range(len(z)):
    data[i][j]=(z[j]-mn)/diff




In [ ]:
data[0][3],data[4][0]

In [ ]:
print(data)

In [ ]:
# data = data.astype(float)
# y = y.astype(int)
data=np.array(data).T
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
print(len(x_train),len(y_train),len(x_test),len(y_test))


In [ ]:
def sigmoid(p):
  val=1/(1 + np.exp(-p))

  return val

In [ ]:
def predict(w,x,i):
  p=w[0]
  for j in range(1,len(w)):
    p+=w[j]*x[i][j-1]
  p=sigmoid(p)
  return p


In [ ]:
def logistic_regg(x,y,m,fs):
  # m is number of examples
  w=[1]*(fs+1) #fs is number of feature
  for j in range(1000):

    totalerror=[0]*(fs+1)

    for i in range(m):
      p=predict(w,x,i)
      error=p-y[i]
      totalerror[0]=totalerror[0]+error
      for k in range(1,len(totalerror)):
        totalerror[k]+= error*x[i][k-1]

    for k in range(len(w)):
      w[k]=w[k]-(0.05*totalerror[k])/m

  return w

In [ ]:
w=logistic_regg(x_train,y_train,len(x_train),len(data[0]))

In [ ]:
w

In [ ]:
ans=[]
for i in w:
  ans.append(i.item())
for i in range(len(ans)):
  print(f"w{i} : ",ans[i])

In [ ]:
def testing(w,x,y):
  m=len(x)
  cp=0
  wp=0
  tp=0
  tn=0
  fp=0
  fn=0
  z=0
  one=0
  for i in range(m):

    p=predict(w,x,i)

    if(p>0.5):
      p=1
    else:
      p=0

    if(p==1 and y[i]==1):
      tp+=1
    elif(p==1 and y[i]==0):
      fp+=1
    elif(p==0 and y[i]==1):
      fn+=1
    else:
      tn+=1

    if(p == y[i]):
      if(p==1):
        one+=1
      cp+=1
    else:
      if(p==1):
        one+=1
      wp+=1
  precision=(tp*100)/(tp+fp)
  recall=(tp*100)/(tp+fn)

  print(tp,tn,fp,fn,one)

  print("Correct Prediction : ", cp)
  print("Wrong Prediction   : ", wp)

  print(f"Accuracy           :  {((cp/m)*100):.2f}%")
  print(f"Precision          :  {precision:.2f}")
  print(f"Recall             :  {recall:.2f}")
  print()
  print()
  # return ans

In [ ]:
ans=testing(w,x_test,y_test)
ans

### **Test out the implementation of Logistic Regression from the scikit-learn package, using saga solver and no regularization penalty**

In [ ]:
from sklearn.linear_model import LogisticRegression

model= LogisticRegression(solver='saga',max_iter=1000, penalty=None)
model.fit(x_train,y_train)

In [ ]:
acc=model.score(x_test,y_test)
print(f"Accuracy without 3 fold : {acc*100:.2f} %")

In [ ]:
from sklearn.metrics import precision_score, recall_score

y_pred = model.predict(x_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy :", acc)
print("Precision:", precision)
print("Recall   :", recall)

####**Cross-validate both the classifiers with 3-folds and print the mean accuracy, precision, and recall for the class 1 (good) for both the classifiers. You may or may not use the scikit-learn implementations for computing these metrics and cross-validation.**

In [ ]:
from sklearn.model_selection import KFold
def k_fold(k,data,y):
  kf=KFold(n_splits=k,shuffle=True)
  return kf

In [ ]:
x=k_fold(3,data,y)
x

In [ ]:
accuracy=[]
for itrain,itest in x.split(data):
  x_train  = data[itrain]
  x_test = data[itest]
  y_train = y[itrain]
  y_test =  y[itest]
  w=logistic_regg(x_train,y_train,len(x_train),len(data[0]))
  acc=testing(w,x_test,y_test)
  accuracy.append(acc)

In [ ]:
accuracy2=[]
p=[]
r=[]
for itrain,itest in x.split(data):
  x_train, x_test = data[itrain], data[itest]
  y_train, y_test = y[itrain], y[itest]
  model= LogisticRegression(solver='saga',max_iter=1000, penalty=None)
  model.fit(x_train,y_train)
  acc=model.score(x_test,y_test)
  y_pred = model.predict(x_test) # Calculate y_pred for the current test set
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  accuracy2.append(acc*100)
  p.append(precision)
  r.append(recall)

In [ ]:
accuracy2,p,r

In [ ]:
from numpy import mean, std
print(mean(accuracy2))
print(mean(p) *100)
print(mean(r) *100)